In [1]:
%matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.ion()

figure, axes = plt.subplots()
input_area = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor="g", facecolor='none')
plotted_x = []
plotted_y = []
draw_points = axes.scatter(plotted_x, plotted_y)

def reset_plot():
    global figure, axes, draw_circle, input_area, plotted_x, plotted_y, draw_points
    figure, axes = plt.subplots()
    draw_circle = plt.Circle((xcent, ycent), r, fill=False)
    input_area = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor="g", facecolor='none')
    plotted_x = []
    plotted_y = []
    draw_points = axes.scatter(plotted_x, plotted_y)

    axes.set_aspect(1)
    axes.add_patch(draw_circle)
    axes.add_patch(input_area)
    axes.add_artist(draw_points)

    plt.title('Spirals')
    plt.draw()

rects = {}
def update_rect(index, bottom_left, height, width, in_class):
    edge_color = "b"
    if in_class:
        edge_color = "r"
    if height < 0.0001 and width < 0.0001:
        height = 0.01
        width = 0.01
    rect = patches.Rectangle(bottom_left, width, height, linewidth=1, edgecolor=edge_color, facecolor='none')
    if index in rects:
        rects[index].remove()
    rects[index] = rect
    axes.add_artist(rect)
    figure.canvas.draw_idle()
    plt.pause(0.01)

def add_point(x,y):
    plotted_x.append(x)
    plotted_y.append(y)
    draw_points.set_offsets(np.c_[plotted_x, plotted_y])
    figure.canvas.draw_idle()
    plt.pause(0.01)

Using matplotlib backend: <object object at 0x10bf6ce10>


In [2]:
import numpy as np
def complement_encode(original_vector: np.array) -> np.array:
    complement = 1-original_vector
    complement_encoded_value = np.concatenate((original_vector,complement))
    return complement_encoded_value

In [3]:
# Replication of section VII. Simulation: Learning to tell two spirals apart, from Carpenter et al., 1992
import numpy as np
from collections import Counter
from fuzzy_artmap import FuzzyArtMap
white_spiral = []
black_spiral = []
for n in range(1,98):
    alpha_n = (np.pi*(n-1)) / 16
    r_n = 0.4*((105-n)/104)
    white_x = r_n * np.sin(alpha_n) + 0.5
    white_y = r_n * np.cos(alpha_n) + 0.5
    black_x = 1 - white_x
    black_y = 1 - white_y
    white_spiral.append([white_x, white_y])
    black_spiral.append([black_x, black_y])

white_spiral = np.array(white_spiral)
black_spiral = np.array(black_spiral)


# test_white_spiral = []
# test_black_spiral = []
# for n in range(1,386):
#     alpha_n = (np.pi*(n-1)) / 16
#     r_n = 0.4*((210-n)/208)
#     white_x = r_n * np.sin(alpha_n) + 0.5
#     white_y = r_n * np.cos(alpha_n) + 0.5
#     black_x = 1 - white_x
#     black_y = 1 - white_y
#     test_white_spiral.append([white_x, white_y])
#     test_black_spiral.append([black_x, black_y])

# test_white_spiral = np.array(test_white_spiral)
# test_black_spiral = np.array(test_black_spiral)

figure, axes = plt.subplots()

# axes.plot(test_white_spiral[:, 0], test_white_spiral[:, 1], "ro")
# axes.plot(test_black_spiral[:, 0], test_black_spiral[:, 1], "ko")

# axes.plot(white_spiral[:, 0], white_spiral[:, 1], "ro")
# axes.plot(black_spiral[:, 0], black_spiral[:, 1], "ko")


# 0 1 == _black_
# 1 0 == _white_


black_vector = np.array([[0], [1]])
white_vector = np.array([[1], [0]])
spiral_fuzzy_artmap = FuzzyArtMap(4, 20, 2, 0.99)
for i, white_point in enumerate(white_spiral):
    white_input = complement_encode(white_point[:,np.newaxis])
    spiral_fuzzy_artmap.train(white_input, white_vector)
    
    black_input = complement_encode(black_spiral[i][:,np.newaxis])
    spiral_fuzzy_artmap.train(black_input, black_vector)

# training_predictions = Counter()
# for i, white_point in enumerate(white_spiral):
#     white_input = complement_encode(white_point[:,np.newaxis])
#     prediction = spiralFuzzy.predict(white_input)

#     correct = np.all(prediction == white_vector)
#     training_predictions.update([correct])
    
#     black_input = complement_encode(black_spiral[i][:,np.newaxis])
#     prediction = spiralFuzzy.predict(black_input)
#     correct = np.all(prediction == black_vector)
#     training_predictions.update([correct])

# print(f"{training_predictions} - {training_predictions[True]/194}")
axes.plot(white_spiral[:, 0], white_spiral[:, 1], "ro")
axes.plot(black_spiral[:, 0], black_spiral[:, 1], "ko")

out_boxes = spiral_fuzzy_artmap.weight_ab[0, :] > spiral_fuzzy_artmap.weight_ab[1, :]
print(np.count_nonzero(spiral_fuzzy_artmap.weight_a[0, :] < 1, 0))
for j in range(spiral_fuzzy_artmap.weight_a.shape[1]):
    is_out_box = out_boxes[j]
    bottom_left = (spiral_fuzzy_artmap.weight_a[0,j], spiral_fuzzy_artmap.weight_a[1,j])
    width = (1-spiral_fuzzy_artmap.weight_a[2,j]) - spiral_fuzzy_artmap.weight_a[0,j]
    height =(1-spiral_fuzzy_artmap.weight_a[3,j]) - spiral_fuzzy_artmap.weight_a[1,j]
    update_rect(j, bottom_left, height, width, is_out_box)


# import math
# def spiral_xy(i, spiral_num):
#     """
#     Create the data for a spiral.

#     Arguments:
#         i runs from 0 to 96
#         spiral_num is 1 or -1
#     """
#     φ = i/16 * math.pi
#     r = 6.5 * ((390 - i)/390)
#     x = (r * math.cos(φ) * spiral_num)/13 + 0.5
#     y = (r * math.sin(φ) * spiral_num)/13 + 0.5
#     return (x, y)

# def spiral(spiral_num):
#     return [spiral_xy(i, spiral_num) for i in range(385)]

# test_white_spiral = np.array(spiral(1))
# test_black_spiral = np.array(spiral(-1))
# figure, axes = plt.subplots()
# axes.plot(test_white_spiral[:, 0], test_white_spiral[:, 1], "ro")
# axes.plot(test_black_spiral[:, 0], test_black_spiral[:, 1], "ko")



# test_predictions = Counter()
# for i, white_point in enumerate(test_white_spiral):
#     white_input = complement_encode(white_point[:,np.newaxis])
#     prediction = spiralFuzzy.predict(white_input)

#     correct = np.all(prediction == white_vector)
#     test_predictions.update([correct])
    
#     black_input = complement_encode(test_black_spiral[i][:,np.newaxis])
#     prediction = spiralFuzzy.predict(black_input)
#     correct = np.all(prediction == black_vector)
#     test_predictions.update([correct])

# print(f"{test_predictions} - {test_predictions[True]/(test_predictions[True] + test_predictions[False])}")

# axes.plot(test_white_spiral[:, 0], test_white_spiral[:, 1], "ro")
# axes.plot(test_black_spiral[:, 0], test_black_spiral[:, 1], "ko")

# out_boxes = spiralFuzzy.weight_ab[0, :] > spiralFuzzy.weight_ab[1, :]
# print(np.count_nonzero(spiralFuzzy.weight_a[0, :] < 1, 0))
# for j in range(spiralFuzzy.weight_a.shape[1]):
#     is_out_box = out_boxes[j]
#     bottom_left = (spiralFuzzy.weight_a[0,j], spiralFuzzy.weight_a[1,j])
#     width = (1-spiralFuzzy.weight_a[2,j]) - spiralFuzzy.weight_a[0,j]
#     height =(1-spiralFuzzy.weight_a[3,j]) - spiralFuzzy.weight_a[1,j]
#     update_rect(j, bottom_left, height, width, is_out_box)
    
# plt.show()

182
